## Code for Looking away

In [1]:
import cv2
import mediapipe as mp
import numpy as np

# Setup MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, refine_landmarks=True, max_num_faces=1)
drawing = mp.solutions.drawing_utils

# Yaw calibration vars
calibration_frames = 30
yaw_sum = 0
frame_count = 0
calibrated_yaw = 0
calibrated = False

# Start webcam
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    h, w, _ = frame.shape
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = face_mesh.process(rgb)

    label = "Looking Straight"

    if result.multi_face_landmarks:
        face = result.multi_face_landmarks[0]
        drawing.draw_landmarks(frame, face, mp_face_mesh.FACEMESH_TESSELATION)
        lm = face.landmark

        # 3D coordinates for yaw estimation
        nose = np.array([lm[1].x, lm[1].y, lm[1].z])
        left_eye = np.array([lm[33].x, lm[33].y, lm[33].z])
        right_eye = np.array([lm[263].x, lm[263].y, lm[263].z])
        eye_mid = (left_eye + right_eye) / 2
        face_vector = nose - eye_mid

        # Yaw angle in degrees
        yaw_angle = -np.arctan2(face_vector[0], face_vector[2]) * 180 / np.pi

        # Visualize face direction
        cx, cy = int(lm[1].x * w), int(lm[1].y * h)
        dx = int(150 * face_vector[0])
        dy = int(150 * face_vector[1])
        cv2.arrowedLine(frame, (cx, cy), (cx + dx, cy + dy), (0, 255, 0), 2)

        # Calibration
        if not calibrated:
            yaw_sum += yaw_angle
            frame_count += 1
            cv2.putText(frame, "Calibrating... Look straight", (20, 460),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)
            if frame_count == calibration_frames:
                calibrated_yaw = yaw_sum / calibration_frames
                calibrated = True
                print(f"✅ Calibrated yaw: {calibrated_yaw:.2f}")
        else:
            corrected_yaw = yaw_angle - calibrated_yaw
            cv2.putText(frame, f"Yaw: {corrected_yaw:.1f}", (20, 100),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (200, 255, 200), 2)
            if abs(corrected_yaw) > 20:
                label = "Looking Away"
            else:
                label = "Looking Straight"
    else:
        label = "No Face Detected"

    # Draw final label
    cv2.putText(frame, label, (20, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 3)

    cv2.imshow("Looking Away Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


✅ Calibrated yaw: 178.46
